In [2]:
#Загружаем бибилиотеку pandas
import pandas as pd

In [53]:
#ЗАДАНИЕ 1

#Читаем файл со списком логов
log = pd.read_csv('visit_log.csv', sep=';')

#Если значение столбца traffic_source равно yandex или google, то в столбец source_type ставим значение organic
log.loc[(log['traffic_source'] == 'yandex')|(log['traffic_source'] == 'google'), 'source_type'] = 'organic'
#Если значение столбца traffic_source равно paid и email, при этом значение столбца region равно Russia,
#то в столбец source_type ставим значение ad
log.loc[((log['traffic_source'] == 'paid')|(log['traffic_source'] == 'email'))&(log['region'] == 'Russia'), 'source_type'] = 'ad'
#Если значение столбца traffic_source равно paid и email, при этом значение столбца region не равно Russia,
#то в столбец source_type ставим значение other
log.loc[((log['traffic_source'] == 'paid')|(log['traffic_source'] == 'email'))&(log['region'] != 'Russia'), 'source_type'] = 'other'
#Все остальные варианты берем из столбца traffic_source без изменений
log.loc[pd.isnull(log['source_type']), 'source_type'] = log.loc[pd.isnull(log['source_type']), 'traffic_source']

#Проверка результата
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [52]:
#ЗАДАНИЕ 2

#Читаем файл с url страниц новостного сайта
urls = pd.read_csv('URLs.txt')

#Необходимо отфильтровать по адресам страниц с текстами новостей
#Использем метод str.contains и регулярное выражение 
urls[urls['url'].str.contains(r'/\d{8}-', regex = True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [51]:
#ЗАДАНИЕ 3

#Читаем файл с оценками фильмов
ratings = pd.read_csv('ml-latest-small/ratings.csv')

#Групируем по пользователям userId и считаем количество оценок для каждого пользователя,
all_users_counts = ratings.groupby('userId').count()
#В полученном результате берём пользователей с количеством оценок больше 100
hundred_users_counts = all_user_counts[all_user_counts['movieId'] >= 100]
#Берём только столбец с userId в виде датафрейма - это нужно для работы функции
result_users_id = hundred_user_counts.reset_index()['userId'].to_frame()

#Функция расчёта времени жизни пользователя
#Принимает на вход userId и весь датафрейм с рейтингами
#Возвращает время жизни пользователя как разницу между макс и мин значениями timestamp
def lifetime(user_id, ratings_all):
    #Берём строки по нашему userId и столбец timestamp
    timestamp_all = ratings.loc[ratings['userId'] == user_id, 'timestamp']
    #Возвращаем разницу между максимальным и минимальным значениями timestamp
    return timestamp_all.max() - timestamp_all.min()

#Добавляем время жизни в столбец lifetime
result_users_id['lifetime'] = result_users_id['userId'].apply(lifetime, ratings_all = ratings)
result_users_id.head()

,userId,lifetime
0,4,203560
1,5,2101
2,8,85187
3,15,471393496
4,17,8053


In [21]:
#ЗАДАНИЕ 4

#Дана статистика услуг перевозок клиентов компании по типам:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

#Таблица с тремя типами выручки для каждого client_id без указания адреса клиента
#Для сохранения всех результатов исользуем outer join
result = pd.merge(pd.merge(rzd, auto, how = 'outer', on = 'client_id'), air, how = 'outer', on = 'client_id')
#Таблица с указанием адресов 
result_with_address = result.merge(client_base, how = 'left', on = 'client_id')
result_with_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
